In [1]:
import pandas as pd
import numpy as np
import itertools
import copy
import requests
import datetime
from datetime import date
import math
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
from espn_api.basketball import League
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)
display(HTML("<style>.container { width:90% !important; }</style>"))

In [183]:
import operator
import functools
functools.reduce(mul, (3, 5, 7, 11, 13, 17, 19)) 

4849845

In [2]:
acronym_to_team_dict = {'ATL': 'Atlanta Hawks',
                        'BOS': 'Boston Celtics',
                        'BKN': 'Brooklyn Nets',
                        'CHA': 'Charlotte Hornets',
                        'CHI': 'Chicago Bulls',
                        'CLE': 'Cleveland Cavaliers',
                        'DAL': 'Dallas Mavericks',
                        'DEN': 'Denver Nuggets',
                        'DET': 'Detroit Pistons',
                        'GSW': 'Golden State Warriors',
                        'HOU': 'Houston Rockets',
                        'IND': 'Indiana Pacers',
                        'LAC': 'Los Angeles Clippers',
                        'LAL': 'Los Angeles Lakers',
                        'MEM': 'Memphis Grizzlies',
                        'MIA': 'Miami Heat',
                        'MIL': 'Milwaukee Bucks',
                        'MIN': 'Minnesota Timberwolves',
                        'NOP': 'New Orleans Pelicans',
                        'NYK': 'New York Knicks',
                        'OKC': 'Oklahoma City Thunder',
                        'ORL': 'Orlando Magic',
                        'PHL': 'Philadelphia 76ers',
                        'PHX': 'Phoenix Suns',
                        'POR': 'Portland Trail Blazers',
                        'SAC': 'Sacramento Kings',
                        'SAS': 'San Antonio Spurs',
                        'TOR': 'Toronto Raptors',
                        'UTA': 'Utah Jazz',
                        'WAS': 'Washignton Wizards'
                        }

team_to_acronym_dict = {'Atlanta Hawks': 'ATL',
                        'Boston Celtics': 'BOS',
                        'Brooklyn Nets': 'BKN',
                        'Charlotte Hornets': 'CHA',
                        'Chicago Bulls': 'CHI',
                        'Cleveland Cavaliers': 'CLE',
                        'Dallas Mavericks': 'DAL',
                        'Denver Nuggets': 'DEN',
                        'Detroit Pistons': 'DET',
                        'Golden State Warriors': 'GSW',
                        'Houston Rockets': 'HOU',
                        'Indiana Pacers': 'IND',
                        'Los Angeles Clippers': 'LAC',
                        'Los Angeles Lakers': 'LAL',
                        'Memphis Grizzlies': 'MEM',
                        'Miami Heat': 'MIA',
                        'Milwaukee Bucks': 'MIL',
                        'Minnesota Timberwolves': 'MIN',
                        'New Orleans Pelicans': 'NOP',
                        'New York Knicks': 'NYK',
                        'Oklahoma City Thunder': 'OKC',
                        'Orlando Magic': 'ORL',
                        'Philadelphia 76ers': 'PHL',
                        'Phoenix Suns': 'PHX',
                        'Portland Trail Blazers': 'POR',
                        'Sacramento Kings': 'SAC',
                        'San Antonio Spurs': 'SAS',
                        'Toronto Raptors': 'TOR',
                        'Utah Jazz': 'UTA',
                        'Washington Wizards': 'WAS'
                       }

In [193]:
def reading_data(file_name):
    """Loads spreadsheet
    Args:
        file_name: path of file
    Returns:
        dataframe of data
    """
    return(pd.read_csv(file_name))

def week_day_teams(schedule_df):   
    week_day_df = {}
    for index, row in schedule_df.iterrows():

        if 'Week ' + str(row['Week']) not in week_day_df.keys():
            week_day_df['Week ' + str(row['Week'])] = {}

        if row['Date'] not in week_day_df['Week ' + str(row['Week'])].keys():
            week_day_df['Week ' + str(row['Week'])][row['Date']] = []

        week_day_df['Week ' + str(row['Week'])][row['Date']].append(team_to_acronym_dict[row['Visitor']])
        week_day_df['Week ' + str(row['Week'])][row['Date']].append(team_to_acronym_dict[row['Home']])

    return week_day_df 

def daily_players(week_num, player_dict, week_day_df):
    week = 'Week ' + str(week_num)
    weekly_dict = week_day_df[week]
    day_dict = {}

    for date, teams in weekly_dict.items():
        for team in teams:
            if team not in player_dict.keys():
                continue

            for player in player_dict[team]:

                if player.injuryStatus == 'OUT':
                    continue
                    
                if date not in day_dict.keys():
                    day_dict[date] = []
                day_dict[date].append(player)
    return day_dict

def team_player_dict_creator(roster):
    player_team_dict = {}
    team_player_dict = {}

    for player in roster:
        player_team_dict[player] = player.proTeam

    for player, team in player_team_dict.items():
        if team not in team_player_dict.keys():
            team_player_dict[team] = [player]
        else:
            team_player_dict[team].append(player)
   
    return(team_player_dict)

def week_projected_points(week_num, week_day_df, team_player_dict):
    position_order_list = ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F', 'UT', 'BE']
    lineup_size = 10
    roster_size = 15
    max_guards_possible = 3
    max_forwards_possible = 3
    guard_positions = ['PG', 'SG']
    foward_positions = ['SF', 'PF']

    day_dict = daily_players(week_num, team_player_dict, week_day_df) 
    day_projected_points = {}

    for day, players in day_dict.items():
        #print(list(itertools.permutations(players)))
        max_players = len(players)
        current_num_players = 0
        roster_spots = {'PG': None, 'SG': None, 'SF': None, 'PF': None, 'C': None, 'G': None, 'F': None, 'UT': []}
        players_list = []
        total_projected_points = 0
        failed = 0
        combination_num = 0
        players_only_position = 0
        max_points = 0

        player_point_dict = {}
        for player in players:
            player_point_dict[player] = player.avg_points

        player_point_dict = {k: v for k, v in sorted(player_point_dict.items(), key=lambda item: item[1], reverse=True)} 
        players_ordered_list = list(player_point_dict.keys())

        position_player_dict = {'PG': [], 'SG': [], 'SF': [], 'PF': [], 'C': [], 'G': [], 'F': []}
        for position in ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F']:
            for player in players_ordered_list:
                if position in player.eligibleSlots:
                    position_player_dict[position].append(player)

        position_only_players = []  # players who are the only player at a position
        one_player_positions = []
        no_one_positon = False  # we will loop again if we have a success
        empty_positions = []  # no players at position
        while no_one_positon == False:

            no_one_positon = True
            for position, players in position_player_dict.items():
                if len(players) == 0 and position not in empty_positions and position not in one_player_positions:
                    empty_positions.append(position) 

                if len(players) == 1 and players[0] not in position_only_players:
                    one_player_positions.append(position)
                    roster_spots[position] = players[0]
                    total_projected_points+=players[0].avg_points
                    players_ordered_list.remove(players[0])
                    position_only_players.append(players[0])
                    players_only_position+=1
                    no_one_positon = False

                    for position2, players2 in position_player_dict.items():
                        if len(players2) == 0:
                            continue
                        if position_only_players[-1] in players2:
                            players2.remove(position_only_players[-1])
                            #position_player_dict[position2].remove(position_only_players[-1])

                if ((position == 'PG' and roster_spots['PG'] != None and roster_spots['SG'] == None and roster_spots['G'] == None and len(position_player_dict['G']) == max_guards_possible-1)
                    or (position == 'SG' and roster_spots['SG'] != None and roster_spots['PG'] == None and roster_spots['G'] == None and len(position_player_dict['G']) == max_guards_possible-1)):
                    if roster_spots['PG'] == None:
                        roster_spots['PG'] = position_player_dict['PG'][0]
                        roster_spots['G'] = position_player_dict['PG'][1]  
                        one_player_positions.append('PG')
                        one_player_positions.append('G')                            
                        total_projected_points+=position_player_dict['PG'][0].avg_points
                        total_projected_points+=position_player_dict['PG'][1].avg_points                      
                        players_ordered_list.remove(position_player_dict['PG'][0])
                        players_ordered_list.remove(position_player_dict['PG'][1])
                        position_only_players.append(position_player_dict['PG'][0])
                        position_only_players.append(position_player_dict['PG'][1])
                        players_only_position+=2
                        no_one_positon = False
                    if roster_spots['SG'] == None:
                        roster_spots['SG'] = position_player_dict['SG'][0]
                        roster_spots['G'] = position_player_dict['SG'][1]  
                        one_player_positions.append('SG')
                        one_player_positions.append('G')                            
                        total_projected_points+=position_player_dict['SG'][0].avg_points
                        total_projected_points+=position_player_dict['SG'][1].avg_points                      
                        players_ordered_list.remove(position_player_dict['SG'][0])
                        players_ordered_list.remove(position_player_dict['SG'][1])
                        position_only_players.append(position_player_dict['SG'][0])
                        position_only_players.append(position_player_dict['SG'][1])
                        players_only_position+=2
                        no_one_positon = False   

                    for position2, players2 in position_player_dict.items():
                        if len(players2) == 0:
                            continue
                        if position_only_players[-1] in players2:
                            players2.remove(position_only_players[-1]) 
                            #position_player_dict[position2].remove(position_only_players[-1])

                        if position_only_players[-2] in players2:
                            players2.remove(position_only_players[-2])
                            #position_player_dict[position2].remove(position_only_players[-2])

                if ((position == 'SF' and roster_spots['SF'] != None and roster_spots['PF'] == None and roster_spots['F'] == None and len(position_player_dict['F']) == max_guards_possible-1)
                    or (position == 'PF' and roster_spots['PF'] != None and roster_spots['SF'] == None and roster_spots['F'] == None and len(position_player_dict['F']) == max_guards_possible-1)):
                    if roster_spots['SF'] == None:
                        roster_spots['SF'] = position_player_dict['SF'][0]
                        roster_spots['F'] = position_player_dict['SF'][1]  
                        one_player_positions.append('SF')
                        one_player_positions.append('F')                            
                        total_projected_points+=position_player_dict['SF'][0].avg_points
                        total_projected_points+=position_player_dict['SF'][1].avg_points                      
                        players_ordered_list.remove(position_player_dict['SF'][0])
                        players_ordered_list.remove(position_player_dict['SF'][1])
                        position_only_players.append(position_player_dict['SF'][0])
                        position_only_players.append(position_player_dict['SF'][1])
                        players_only_position+=2
                        no_one_positon = False
                    if roster_spots['PF'] == None:
                        roster_spots['PF'] = position_player_dict['PF'][0]
                        roster_spots['F'] = position_player_dict['PF'][1]  
                        one_player_positions.append('PF')
                        one_player_positions.append('F')                            
                        total_projected_points+=position_player_dict['PF'][0].avg_points
                        total_projected_points+=position_player_dict['PF'][1].avg_points                      
                        players_ordered_list.remove(position_player_dict['PF'][0])
                        players_ordered_list.remove(position_player_dict['PF'][1])
                        position_only_players.append(position_player_dict['PF'][0])
                        position_only_players.append(position_player_dict['PF'][1])
                        players_only_position+=2
                        no_one_positon = False   

                    for position2, players2 in position_player_dict.items():
                        if len(players2) == 0:
                            continue
                        if position_only_players[-1] in players2:
                            players2.remove(position_only_players[-1]) 

                        if position_only_players[-2] in players2:
                            players2.remove(position_only_players[-2]) 

        num_empty_positions = len(empty_positions)   # Number of positions that will be empty
        """   if 'PG' in empty_positions and 'SG' in empty_positions:
            num_empty_positions+=1
        if 'SF' in empty_positions and 'PF' in empty_positions:
            num_empty_positions+=1
        #if roster_spots['PG'] != None and roster_spots['SG'] != None:   # G position will be empty
        #    print("aaaaaaaaa")
         #   num_empty_positions+=1
        #if roster_spots['SF'] != None and roster_spots['PF'] != None:   # F position will be empty
        #    print("bbbbbbbbbbbbbbbbbbb")
         #   num_empty_positions+=1
        """

        while (current_num_players + players_only_position + num_empty_positions) < min(max_players, lineup_size):
            players_ordered_list_min = []

            if failed > 0:  # If first attempt fails
                print("FAILED " + str(failed))
                if failed == 1:
                    possible_iterations = list(itertools.permutations(players_ordered_list))

                if max_players <= lineup_size:  # If number of players playing is less than or equal to starting spots
                    if failed >= len(possible_iterations):
                        tried_every_combination = 1
                        day_projected_points[day] = max_points
                        print("Tried every Combo")
                        print(day_projected_points)
                        break
                    players_ordered_list_min = possible_iterations[failed]
                else:  # If number of players playing is greater than starting spots
                    if combination_num == 0:  # If have not failed with all attmepts at original list
                        temp_list = players_ordered_list[:lineup_size]  # temporary list
                    if failed <= math.factorial(lineup_size):  # If not failed with all attempts at current list
                        players_ordered_list_min = possible_iterations[failed]
                    else:  # If have failed with all attempts at current list
                        combination_num += 1  # Will increase for each failure
                        temp_list = list(itertools.combinations(players_ordered_list, lineup_size))[combination_num]  # Going to next combination
                        players_ordered_list_min = temp_list
                        failed = 0  # Resetting fails

            else:
                players_ordered_list_min = players_ordered_list[:lineup_size]

            for player in players_ordered_list_min:
                eligible_slots = []

                for pos in position_order_list:
                    if pos in player.eligibleSlots:
                        eligible_slots.append(pos)

                for spot in eligible_slots:

                    if spot in roster_spots.keys() and spot != 'UT' and roster_spots[spot] == None:
                        roster_spots[spot] = player
                        players_list.append(player)
                        total_projected_points+=player.avg_points
                        current_num_players+=1
                        break
                    elif spot == 'UT' and len(roster_spots[spot]) < 3:
                        roster_spots[spot].append(player)
                        players_list.append(player)
                        total_projected_points+=player.avg_points
                        current_num_players+=1
                        break

            if ((current_num_players + players_only_position + num_empty_positions) < max_players and max_players <= lineup_size) or (max_players > lineup_size and (current_num_players + players_only_position + num_empty_positions) < lineup_size):
                current_num_players = 0
                players_list = []
                if max_points < total_projected_points:   # If this iteration performed the best, make a note
                    max_points = total_projected_points

                total_projected_points = 0
                roster_spots = {'PG': None, 'SG': None, 'SF': None, 'PF': None, 'C': None, 'G': None, 'F': None, 'UT': []}
                failed +=1
        if day not in day_projected_points.keys():
            day_projected_points[day] = total_projected_points

    return(day_projected_points, sum(day_projected_points.values()))

def daily_projected_points(week_num, day, team_player_dict, week_day_teams_df):
    print(day)
    position_order_list = ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F', 'UT', 'BE']
    position_prime_number_dict = {'PG': 3, 'SG': 5, 'SF': 7, 'PF': 11, 'C': 13, 'G': 17, 'F': 19}
    lineup_size = 10
    roster_size = 15
    max_guards_possible = 3
    max_forwards_possible = 3
    guard_positions = ['PG', 'SG']
    foward_positions = ['SF', 'PF']
    if day not in daily_players(week_num, team_player_dict, week_day_teams_df).keys():
        return 0
    
    players = daily_players(week_num, team_player_dict, week_day_teams_df)[day] 

    max_players = len(players)
    current_num_players = 0
    roster_spots = {'PG': None, 'SG': None, 'SF': None, 'PF': None, 'C': None, 'G': None, 'F': None, 'UT': []}
    players_list = []
    total_projected_points = 0
    failed = 0
    combination_num = 0
    players_only_position = 0
    max_points = 0
    
    player_point_dict = {}
    for player in players:
        player_point_dict[player] = player.avg_points

    player_point_dict = {k: v for k, v in sorted(player_point_dict.items(), key=lambda item: item[1], reverse=True)} 
    players_ordered_list = list(player_point_dict.keys())
    print(player_point_dict)
    position_player_dict = {'PG': [], 'SG': [], 'SF': [], 'PF': [], 'C': [], 'G': [], 'F': []}
    for position in ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F']:
        for player in players_ordered_list:
            if position in player.eligibleSlots:
                position_player_dict[position].append(player)
    print(position_player_dict)
    position_only_players = []  # players who are the only player at a position
    one_player_positions = []
    no_one_positon = False  # we will loop again if we have a success
    empty_positions = []  # no players at position
    
    player_prime_num_dict = {}
    
    for player in players_ordered_list[:lineup_size]:
        eligible_slots = []
        
        player_prime_num_dict[player] = []

        for pos in ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F']:
            if pos in player.eligibleSlots:
                player_prime_num_dict[player].append(position_prime_number_dict[pos])
    print(players_ordered_list[:lineup_size])
    if len(players_ordered_list[:lineup_size]) > 4:
        found_combo = False
        for combo in itertools.product(*player_prime_num_dict.values()):
            print(combo)
            player_position_score = functools.reduce(mul, combo)
            
            mult = 1
            for pos, l in position_player_dict.items():
                if len(l) == 0:
                    player_position_score*= position_prime_number_dict[pos]
                    
            print(player_position_score)
            for val in [np.prod(list(itertools.combinations([3,5,7,11,13,17,19],len(players_ordered_list[:lineup_size])-3))[num]) for num in range(math.comb(7, len(players_ordered_list[:lineup_size])-3))]:
                
                if  player_position_score%(val) == 0:
                    found_combo = True
                    break
            if found_combo == True:
                break

        if found_combo == False:
            print("No combo Failure")
    print("Here")
    
    while no_one_positon == False:

        no_one_positon = True
        for position, players in position_player_dict.items():
            if len(players) == 0 and position not in empty_positions and position not in one_player_positions:
                empty_positions.append(position) 

            if len(players) == 1 and players[0] not in position_only_players:
                one_player_positions.append(position)
                roster_spots[position] = players[0]
                total_projected_points+=players[0].avg_points
                players_ordered_list.remove(players[0])
                position_only_players.append(players[0])
                players_only_position+=1
                no_one_positon = False

                for position2, players2 in position_player_dict.items():
                    if len(players2) == 0:
                        continue
                    if position_only_players[-1] in players2:
                        players2.remove(position_only_players[-1])
                        #position_player_dict[position2].remove(position_only_players[-1])

            if ((position == 'PG' and roster_spots['PG'] != None and roster_spots['SG'] == None and roster_spots['G'] == None and len(position_player_dict['G']) == max_guards_possible-1)
                or (position == 'SG' and roster_spots['SG'] != None and roster_spots['PG'] == None and roster_spots['G'] == None and len(position_player_dict['G']) == max_guards_possible-1)):
                if roster_spots['PG'] == None:
                    roster_spots['PG'] = position_player_dict['PG'][0]
                    roster_spots['G'] = position_player_dict['PG'][1]  
                    one_player_positions.append('PG')
                    one_player_positions.append('G')                            
                    total_projected_points+=position_player_dict['PG'][0].avg_points
                    total_projected_points+=position_player_dict['PG'][1].avg_points                      
                    players_ordered_list.remove(position_player_dict['PG'][0])
                    players_ordered_list.remove(position_player_dict['PG'][1])
                    position_only_players.append(position_player_dict['PG'][0])
                    position_only_players.append(position_player_dict['PG'][1])
                    players_only_position+=2
                    no_one_positon = False
                if roster_spots['SG'] == None:
                    roster_spots['SG'] = position_player_dict['SG'][0]
                    roster_spots['G'] = position_player_dict['SG'][1]  
                    one_player_positions.append('SG')
                    one_player_positions.append('G')                            
                    total_projected_points+=position_player_dict['SG'][0].avg_points
                    total_projected_points+=position_player_dict['SG'][1].avg_points                      
                    players_ordered_list.remove(position_player_dict['SG'][0])
                    players_ordered_list.remove(position_player_dict['SG'][1])
                    position_only_players.append(position_player_dict['SG'][0])
                    position_only_players.append(position_player_dict['SG'][1])
                    players_only_position+=2
                    no_one_positon = False   

                for position2, players2 in position_player_dict.items():
                    if len(players2) == 0:
                        continue
                    if position_only_players[-1] in players2:
                        players2.remove(position_only_players[-1]) 
                        #position_player_dict[position2].remove(position_only_players[-1])

                    if position_only_players[-2] in players2:
                        players2.remove(position_only_players[-2])
                        #position_player_dict[position2].remove(position_only_players[-2])

            if ((position == 'SF' and roster_spots['SF'] != None and roster_spots['PF'] == None and roster_spots['F'] == None and len(position_player_dict['F']) == max_guards_possible-1)
                or (position == 'PF' and roster_spots['PF'] != None and roster_spots['SF'] == None and roster_spots['F'] == None and len(position_player_dict['F']) == max_guards_possible-1)):
                if roster_spots['SF'] == None:
                    roster_spots['SF'] = position_player_dict['SF'][0]
                    roster_spots['F'] = position_player_dict['SF'][1]  
                    one_player_positions.append('SF')
                    one_player_positions.append('F')                            
                    total_projected_points+=position_player_dict['SF'][0].avg_points
                    total_projected_points+=position_player_dict['SF'][1].avg_points                      
                    players_ordered_list.remove(position_player_dict['SF'][0])
                    players_ordered_list.remove(position_player_dict['SF'][1])
                    position_only_players.append(position_player_dict['SF'][0])
                    position_only_players.append(position_player_dict['SF'][1])
                    players_only_position+=2
                    no_one_positon = False
                if roster_spots['PF'] == None:
                    roster_spots['PF'] = position_player_dict['PF'][0]
                    roster_spots['F'] = position_player_dict['PF'][1]  
                    one_player_positions.append('PF')
                    one_player_positions.append('F')                            
                    total_projected_points+=position_player_dict['PF'][0].avg_points
                    total_projected_points+=position_player_dict['PF'][1].avg_points                      
                    players_ordered_list.remove(position_player_dict['PF'][0])
                    players_ordered_list.remove(position_player_dict['PF'][1])
                    position_only_players.append(position_player_dict['PF'][0])
                    position_only_players.append(position_player_dict['PF'][1])
                    players_only_position+=2
                    no_one_positon = False   

                for position2, players2 in position_player_dict.items():
                    if len(players2) == 0:
                        continue
                    if position_only_players[-1] in players2:
                        players2.remove(position_only_players[-1]) 

                    if position_only_players[-2] in players2:
                        players2.remove(position_only_players[-2]) 

    num_empty_positions = len(empty_positions)   # Number of positions that will be empty
    """   if 'PG' in empty_positions and 'SG' in empty_positions:
        num_empty_positions+=1
    if 'SF' in empty_positions and 'PF' in empty_positions:
        num_empty_positions+=1
    #if roster_spots['PG'] != None and roster_spots['SG'] != None:   # G position will be empty
    #    print("aaaaaaaaa")
     #   num_empty_positions+=1
    #if roster_spots['SF'] != None and roster_spots['PF'] != None:   # F position will be empty
    #    print("bbbbbbbbbbbbbbbbbbb")
     #   num_empty_positions+=1
    """
    print("current_num " + str(current_num_players))
    print("players_only_position " + str(players_only_position))
    print("num_empty_positions " + str(num_empty_positions))
    print("max player " + str(max_players))
    while (current_num_players + players_only_position + num_empty_positions) < min(max_players, lineup_size):
        players_ordered_list_min = []

        if failed > 0:  # If first attempt fails
            print("FAILED " + str(failed))
            if failed == 1:
                #possible_iterations = list(itertools.permutations(players_ordered_list))
                
                for possible_iterations in list(itertools.permutations(players_ordered_list)):
                    
                    players_list, roster_spots, current_num_players, total_projected_points = player_organizer(players_ordered_list_min, position_order_list, roster_sptos)
        
                    print("roster_spots")
                    print(roster_spots)
                    if ((current_num_players + players_only_position + num_empty_positions) < max_players and max_players <= lineup_size) or (max_players > lineup_size and (current_num_players + players_only_position + num_empty_positions) < lineup_size):
                        current_num_players = 0
                        players_list = []
                        if max_points < total_projected_points:   # If this iteration performed the best, make a note
                            max_points = total_projected_points

                        total_projected_points = 0
                        roster_spots = {'PG': None, 'SG': None, 'SF': None, 'PF': None, 'C': None, 'G': None, 'F': None, 'UT': []}
                        failed +=1
                        print("FAILED " + str(failed))
                    
                    else:
                        return(total_projected_points)
                    
                    
                if max_players <= lineup_size:  # If number of players playing is less than or equal to starting spots
                    if failed >= len(possible_iterations):
                        tried_every_combination = 1
                        print("Tried every Combo")
                        break
                else:
                    if combination_num == 0:  # If have not failed with all attmepts at original list
                        temp_list = players_ordered_list[:lineup_size]  # temporary list

                    if failed <= math.factorial(lineup_size):  # If not failed with all attempts at current list
                        players_ordered_list_min = possible_iterations[failed]
                    else:  # If have failed with all attempts at current list
                        combination_num += 1  # Will increase for each failure
                        temp_list = list(itertools.combinations(players_ordered_list, lineup_size))[combination_num]  # Going to next combination
                        players_ordered_list_min = temp_list
                        failed = 0  # Resetting fails
                    

            if max_players <= lineup_size:  # If number of players playing is less than or equal to starting spots
                if failed >= len(possible_iterations):
                    tried_every_combination = 1
                    print("Tried every Combo")
                    break
                players_ordered_list_min = possible_iterations[failed]
            else:  # If number of players playing is greater than starting spots
                if combination_num == 0:  # If have not failed with all attmepts at original list
                    temp_list = players_ordered_list[:lineup_size]  # temporary list
                if failed <= math.factorial(lineup_size):  # If not failed with all attempts at current list
                    players_ordered_list_min = possible_iterations[failed]
                else:  # If have failed with all attempts at current list
                    combination_num += 1  # Will increase for each failure
                    temp_list = list(itertools.combinations(players_ordered_list, lineup_size))[combination_num]  # Going to next combination
                    players_ordered_list_min = temp_list
                    failed = 0  # Resetting fails

        else:
            players_ordered_list_min = players_ordered_list[:lineup_size]

        players_list, roster_spots, current_num_players, total_projected_points = player_organizer(players_ordered_list_min, position_order_list, roster_spots)
        
        print("roster_spots")
        print(roster_spots)
        if ((current_num_players + players_only_position + num_empty_positions) < max_players and max_players <= lineup_size) or (max_players > lineup_size and (current_num_players + players_only_position + num_empty_positions) < lineup_size):
            print(roster_spots)
            print(players_list)
            
            aaaaa = bubble_sorter()
            
            current_num_players = 0
            players_list = []
            if max_points < total_projected_points:   # If this iteration performed the best, make a note
                max_points = total_projected_points

            total_projected_points = 0
            roster_spots = {'PG': None, 'SG': None, 'SF': None, 'PF': None, 'C': None, 'G': None, 'F': None, 'UT': []}
            failed +=1


    return(total_projected_points)

def player_organizer(list_players, position_order_list, team_roster_spots):
    
    projected_points = 0
    spots_filled = 0
    
    for player in list_players:
        eligible_slots = []

        for pos in position_order_list:
            if pos in player.eligibleSlots:
                eligible_slots.append(pos)

        for spot in eligible_slots:

            if spot in team_roster_spots.keys() and spot != 'UT' and team_roster_spots[spot] == None:
                team_roster_spots[spot] = player
                list_players.append(player)
                projected_points+=player.avg_points
                spots_filled+=1
                break
            elif spot == 'UT' and len(team_roster_spots[spot]) < 3:
                team_roster_spots[spot].append(player)
                list_players.append(player)
                projected_points+=player.avg_points
                spots_filled+=1
                break
                
    return list_players, team_roster_spots, spots_filled, projected_points



In [4]:
file_name = 'C:\\Users\\Jack\\Basketball Project\\NBA_Schedule.csv'
schedule_df = reading_data(file_name) # Retrieving Spreadsheet
league = League(league_id=1762898804, year=2023, espn_s2 = 'AEAkbmf6ntGyDFU%2BZ5MICm2WvwmwQ7SsLFomcaskbGz8VksoIPAE1D3MYbsTpcyiNBklJETozxCt1D0n3U7yat2RFOSQrW%2FqZdbWbsdxVWuRfOGKCfOfLrdDSYm2VsduCiWS0c5JLZ2zXzCg1UQqzReuFgIK8KLGyLj9%2FNe2IGgEgPqjwcqEp6PSBb6EssdzMJmuQMRgWtWIBE%2BRX1Ii%2FqLJTRTfLOchv6DK2%2FY%2BitmWtiY4Oa2n6gyF0Acup9V34nFNYB8IKmAZLpZqz3QFlTPw', swid='{A98C2FBC-429C-481F-8D11-8EDAE4526062}', debug=False)
free_agents = league.free_agents(size = 250)

In [216]:
list(itertools.combinations(range(4), 5))

[]

In [274]:
def daily_projected_points(week_num, day, team_player_dict, week_day_teams_df):
    print("Made it")
    position_prime_number_dict = {'PG': 3, 'SG': 5, 'SF': 7, 'PF': 11, 'C': 13, 'G': 17, 'F': 19}
    lineup_size = 10
    utility_spots = 3

    if day not in daily_players(week_num, team_player_dict, week_day_teams_df).keys():
        return 0
    else:
        players = daily_players(week_num, team_player_dict, week_day_teams_df)[day] 
        print('day of games ' + str(day))
        total_projected_points = 0


        player_point_dict = {}
        for player in players:
            player_point_dict[player] = player.avg_points

        player_point_dict = {k: v for k, v in sorted(player_point_dict.items(), key=lambda item: item[1], reverse=True)} 
        players_ordered_list = list(player_point_dict.keys())
        print(players_ordered_list)

        position_player_dict = {'PG': [], 'SG': [], 'SF': [], 'PF': [], 'C': [], 'G': [], 'F': []}
        for position in ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F']:
            for player in players_ordered_list:
                if position in player.eligibleSlots:
                    position_player_dict[position].append(player)

        empty_positions = []
        for pos, l in position_player_dict.items():   # Finding positions with no players so we can ignore them
            if len(l) == 0:
                empty_positions.append(pos)           
        prime_number_list = [v for k, v in position_prime_number_dict.items() if k not in empty_positions]    # Removing prme numbers associated with empty positions           
        print(len(players_ordered_list)-len(empty_positions))
        for attempted_player_count in range(min(lineup_size-len(empty_positions), len(players_ordered_list)), utility_spots+1, -1):
            for player_combo in itertools.combinations(players_ordered_list, attempted_player_count):  # For all combinations of possible players 
                player_prime_num_dict = {}   # For each player, getting prime numbers associated with their positions

                for player in player_combo[:lineup_size]:
                    player_prime_num_dict[player] = []

                    for pos in ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F']:
                        if pos in player.eligibleSlots:
                            player_prime_num_dict[player].append(position_prime_number_dict[pos])

                #print(player_combo[:lineup_size])
                found_combo = False
                for combo in itertools.product(*player_prime_num_dict.values()):  # Combinations of player prime numbers
                    player_position_score = functools.reduce(mul, combo)   # Multiplying numbers in the given combination
                    #print(combo)
                    #print(player_position_score)

                    for val in ([np.prod(list(itertools.combinations(prime_number_list,attempted_player_count-utility_spots))[num]) 
                                  for num in range(math.comb(len(prime_number_list), attempted_player_count-utility_spots))]):   # Possible combinations of products of prime numbers of availble position
                        #print('val' + str(math.comb(len(prime_number_list), attempted_player_count-utility_spots)))  
                        if  player_position_score%val == 0:   # Success if the product of player prime numbers is divisible by the product of the position prime numbers
                            found_combo = True
                            print("Found Combo")            
                            total_projected_points = 0
                            for player in player_combo:   # After success, finding total projected points
                                total_projected_points += player_point_dict[player]
                            print("total_projected_points " + str(total_projected_points))    
                            return total_projected_points
                            #print("total_projected_points " + str(total_projected_points))
                        #if found_combo == True:
                            #break
                    #if found_combo == True:
                       # break

                #if found_combo == False:
            print("No combo Failure")
            ##print("Here")

        if len(players_ordered_list) <= utility_spots+1:
            for player in players_ordered_list:   # After success, finding total projected points
                total_projected_points += player_point_dict[player]
            print("total_projected_points " + str(total_projected_points)) 
            return total_projected_points
    print('returning 0')    
    return 0

In [289]:
week = 9
print("week " + str(week))
print("team " + str(league_team))
team = league.teams[5]
orig_roster = copy.copy(team.roster)
week_day_df = week_day_teams(schedule_df)
orig_weekly_points_dict, orig_weekly_points = week_projected_points(week, week_day_df, team_player_dict_creator(orig_roster))
print("orig_weekly_points " + str(orig_weekly_points))

day_of_week = 0
prior_points = 0

player_value_dict = {}
for day in week_day_df['Week ' + str(week)].keys(): 
    print(day)
    #if datetime.datetime.strptime(day, '%m/%d/%y') < datetime.datetime.today():
        #continue
    print("Making changes prior to " + day)
    if day_of_week > 0:
        prior_points = 0
        for prev_day in week_day_df['Week ' + str(week)].keys():
            if prev_day >= day:
                break
            prior_points+=daily_projected_points(week, prev_day, team_player_dict_creator(orig_roster), week_day_df)

    day_of_week+=1
    print("Points prior to day " + str(prior_points))

    for player in team.roster:
        if player.avg_points > 25:
            continue

        else:
            if player not in player_value_dict.keys():
                player_value_dict[player] = {}
            for free_agent in free_agents:
                if free_agent.injuryStatus != 'ACTIVE' or free_agent.avg_points < 18:
                    continue

                else:
                    print("free agent " + free_agent.name)
                    print('dropping player ' + player.name)
                    temp_roster = copy.copy(orig_roster)
                    temp_roster.remove(player)
                    temp_roster.append(free_agent)
                    temp_roster_dict = team_player_dict_creator(temp_roster)
                    additional_points = 0
                    for temp_day in week_day_df['Week ' + str(week)].keys():
                        if temp_day < day:
                            continue

                        additional_points+=daily_projected_points(week, temp_day, temp_roster_dict, week_day_df)

                    total_weekly_additional_points =  prior_points + additional_points - orig_weekly_points   
                    print('ttw ' + str(total_weekly_additional_points))
                    if total_weekly_additional_points > 10:
                        if free_agent not in player_value_dict[player].keys():
                            player_value_dict[player][free_agent] = [day, total_weekly_additional_points]
                        elif total_weekly_additional_points > player_value_dict[player][free_agent][1]:
                            player_value_dict[player][free_agent] = [day, total_weekly_additional_points]

                        player_value_dict[player] = {k: v for k, v in sorted(player_value_dict[player].items(), key=lambda item: item[1][1], reverse = True)} 
player_value_dict

week 9
team 3
orig_weekly_points 1284.12
12/12/22
Making changes prior to 12/12/22
Points prior to day 0
free agent Keegan Murray
dropping player Malik Beasley
Made it
day of games 12/12/22
[Player(Kyle Kuzma), Player(Keldon Johnson), Player(Tre Jones), Player(Royce O'Neale), Player(Reggie Jackson), Player(Dorian Finney-Smith)]
4
Found Combo
total_projected_points 167.24
Made it
day of games 12/13/22
[Player(Russell Westbrook), Player(Jordan Clarkson), Player(Jonas Valanciunas), Player(Keegan Murray)]
3
total_projected_points 113.68
Made it
day of games 12/14/22
[Player(Kyle Kuzma), Player(O.G. Anunoby), Player(Keldon Johnson), Player(Scottie Barnes), Player(Tre Jones), Player(Bol Bol), Player(Reggie Jackson), Player(Kentavious Caldwell-Pope), Player(Dorian Finney-Smith), Player(Keegan Murray)]
10
Found Combo
total_projected_points 275.15
Made it
day of games 12/15/22
[Player(Jordan Clarkson), Player(Jonas Valanciunas), Player(Reggie Jackson)]
0
total_projected_points 84.34
Made it
day

Found Combo
total_projected_points 217.75
Made it
day of games 12/17/22
[Player(Kyle Kuzma), Player(Keldon Johnson), Player(Jordan Clarkson), Player(Jonas Valanciunas), Player(Tre Jones), Player(Reggie Jackson), Player(Dorian Finney-Smith)]
7
Found Combo
total_projected_points 201.39000000000001
Made it
day of games 12/18/22
[Player(Kyle Kuzma), Player(O.G. Anunoby), Player(Russell Westbrook), Player(Scottie Barnes), Player(Bol Bol), Player(Royce O'Neale), Player(Alec Burks), Player(Kentavious Caldwell-Pope)]
8
Found Combo
total_projected_points 234.68999999999997
ttw -5.789999999999736
free agent Jose Alvarado
dropping player Malik Beasley
Made it
day of games 12/12/22
[Player(Kyle Kuzma), Player(Keldon Johnson), Player(Tre Jones), Player(Royce O'Neale), Player(Reggie Jackson), Player(Dorian Finney-Smith)]
4
Found Combo
total_projected_points 167.24
Made it
day of games 12/13/22
[Player(Russell Westbrook), Player(Jordan Clarkson), Player(Jonas Valanciunas), Player(Jose Alvarado)]
1
to

Found Combo
total_projected_points 234.58999999999997
ttw -4.079999999999927
free agent Ayo Dosunmu
dropping player Reggie Jackson
Made it
day of games 12/12/22
[Player(Kyle Kuzma), Player(Keldon Johnson), Player(Tre Jones), Player(Royce O'Neale), Player(Dorian Finney-Smith)]
3
Found Combo
total_projected_points 144.09
Made it
day of games 12/13/22
[Player(Russell Westbrook), Player(Jordan Clarkson), Player(Jonas Valanciunas), Player(Malik Beasley)]
1
total_projected_points 118.77
Made it
day of games 12/14/22
[Player(Kyle Kuzma), Player(O.G. Anunoby), Player(Keldon Johnson), Player(Scottie Barnes), Player(Tre Jones), Player(Bol Bol), Player(Kentavious Caldwell-Pope), Player(Ayo Dosunmu), Player(Dorian Finney-Smith)]
9
Found Combo
total_projected_points 253.41
Made it
day of games 12/15/22
[Player(Jordan Clarkson), Player(Jonas Valanciunas), Player(Malik Beasley)]
-1
total_projected_points 85.35
Made it
day of games 12/16/22
[Player(O.G. Anunoby), Player(Russell Westbrook), Player(Scot

Found Combo
total_projected_points 212.45999999999998
ttw -29.359999999999673
free agent Jeremiah Robinson-Earl
dropping player Reggie Jackson
Made it
day of games 12/12/22
[Player(Kyle Kuzma), Player(Keldon Johnson), Player(Tre Jones), Player(Royce O'Neale), Player(Dorian Finney-Smith), Player(Jeremiah Robinson-Earl)]
5
Found Combo
total_projected_points 163.21
Made it
day of games 12/13/22
[Player(Russell Westbrook), Player(Jordan Clarkson), Player(Jonas Valanciunas), Player(Malik Beasley)]
1
total_projected_points 118.77
Made it
day of games 12/14/22
[Player(Kyle Kuzma), Player(O.G. Anunoby), Player(Keldon Johnson), Player(Scottie Barnes), Player(Tre Jones), Player(Bol Bol), Player(Kentavious Caldwell-Pope), Player(Dorian Finney-Smith), Player(Jeremiah Robinson-Earl)]
9
Found Combo
total_projected_points 252.05
Made it
day of games 12/15/22
[Player(Jordan Clarkson), Player(Jonas Valanciunas), Player(Malik Beasley)]
-1
total_projected_points 85.35
Made it
day of games 12/16/22
[Playe

Found Combo
total_projected_points 236.9
Made it
day of games 12/15/22
[Player(Jordan Clarkson), Player(Jonas Valanciunas), Player(Malik Beasley), Player(Reggie Jackson), Player(Santi Aldama)]
4
Found Combo
total_projected_points 130.9
Made it
day of games 12/16/22
[Player(O.G. Anunoby), Player(Russell Westbrook), Player(Scottie Barnes), Player(Bol Bol), Player(Royce O'Neale), Player(Kentavious Caldwell-Pope)]
6
Found Combo
total_projected_points 176.34
Made it
day of games 12/17/22
[Player(Kyle Kuzma), Player(Keldon Johnson), Player(Jordan Clarkson), Player(Jonas Valanciunas), Player(Tre Jones), Player(Malik Beasley), Player(Reggie Jackson), Player(Santi Aldama)]
8
Found Combo
total_projected_points 228.77
Made it
day of games 12/18/22
[Player(Kyle Kuzma), Player(O.G. Anunoby), Player(Russell Westbrook), Player(Scottie Barnes), Player(Bol Bol), Player(Royce O'Neale), Player(Kentavious Caldwell-Pope)]
7
Found Combo
total_projected_points 212.45999999999998
ttw -9.519999999999754
free a

...

day of games 12/16/22
[Player(O.G. Anunoby), Player(Russell Westbrook), Player(Scottie Barnes), Player(Bol Bol), Player(Royce O'Neale), Player(Andre Drummond), Player(Dorian Finney-Smith)]
6
Found Combo
total_projected_points 194.25
Made it
day of games 12/17/22
[Player(Kyle Kuzma), Player(Keldon Johnson), Player(Jordan Clarkson), Player(Jonas Valanciunas), Player(Tre Jones), Player(Malik Beasley), Player(Reggie Jackson), Player(Dorian Finney-Smith)]
8
Found Combo
total_projected_points 225.55
Made it
day of games 12/18/22
[Player(Kyle Kuzma), Player(O.G. Anunoby), Player(Russell Westbrook), Player(Scottie Barnes), Player(Bol Bol), Player(Royce O'Neale), Player(Andre Drummond)]
6
Found Combo
total_projected_points 211.19
ttw -2.5399999999999636
free agent Jarred Vanderbilt
dropping player Kentavious Caldwell-Pope
Made it
day of games 12/16/22
[Player(O.G. Anunoby), Player(Russell Westbrook), Player(Scottie Barnes), Player(Bol Bol), Player(Royce O'Neale), Player(Dorian Finney-Smith)]
5


ttw -5.039999999999964
free agent Moritz Wagner
dropping player Malik Beasley
Made it
day of games 12/17/22
[Player(Kyle Kuzma), Player(Keldon Johnson), Player(Jordan Clarkson), Player(Jonas Valanciunas), Player(Tre Jones), Player(Reggie Jackson), Player(Dorian Finney-Smith)]
7
Found Combo
total_projected_points 201.39000000000001
Made it
day of games 12/18/22
[Player(Kyle Kuzma), Player(O.G. Anunoby), Player(Russell Westbrook), Player(Scottie Barnes), Player(Bol Bol), Player(Royce O'Neale), Player(Kentavious Caldwell-Pope), Player(Moritz Wagner)]
8
Found Combo
total_projected_points 233.80999999999997
ttw -2.8099999999999454
free agent Gabe Vincent
dropping player Malik Beasley
Made it
day of games 12/17/22
[Player(Kyle Kuzma), Player(Keldon Johnson), Player(Jordan Clarkson), Player(Jonas Valanciunas), Player(Tre Jones), Player(Reggie Jackson), Player(Dorian Finney-Smith), Player(Gabe Vincent)]
8
Found Combo
total_projected_points 219.51000000000002
Made it
day of games 12/18/22
[Play

8
Found Combo
total_projected_points 225.87
Made it
day of games 12/18/22
[Player(Kyle Kuzma), Player(O.G. Anunoby), Player(Russell Westbrook), Player(Scottie Barnes), Player(Bol Bol), Player(Royce O'Neale), Player(Kentavious Caldwell-Pope)]
7
Found Combo
total_projected_points 212.45999999999998
ttw 0.31999999999993634
free agent Jalen Williams
dropping player Dorian Finney-Smith
Made it
day of games 12/17/22
[Player(Kyle Kuzma), Player(Keldon Johnson), Player(Jordan Clarkson), Player(Jonas Valanciunas), Player(Tre Jones), Player(Malik Beasley), Player(Reggie Jackson), Player(Jalen Williams)]
8
Found Combo
total_projected_points 225.94
Made it
day of games 12/18/22
[Player(Kyle Kuzma), Player(O.G. Anunoby), Player(Russell Westbrook), Player(Scottie Barnes), Player(Bol Bol), Player(Royce O'Neale), Player(Kentavious Caldwell-Pope)]
7
Found Combo
total_projected_points 212.45999999999998
ttw 0.38999999999987267
free agent Zach Collins
dropping player Dorian Finney-Smith
Made it
day of ga

Found Combo
total_projected_points 212.45999999999998
ttw 0.0
free agent Tari Eason
dropping player Reggie Jackson
Made it
day of games 12/18/22
[Player(Kyle Kuzma), Player(O.G. Anunoby), Player(Russell Westbrook), Player(Scottie Barnes), Player(Bol Bol), Player(Royce O'Neale), Player(Kentavious Caldwell-Pope)]
7
Found Combo
total_projected_points 212.45999999999998
ttw 0.0
free agent Jalen Duren
dropping player Reggie Jackson
Made it
day of games 12/18/22
[Player(Kyle Kuzma), Player(O.G. Anunoby), Player(Russell Westbrook), Player(Scottie Barnes), Player(Bol Bol), Player(Royce O'Neale), Player(Kentavious Caldwell-Pope), Player(Jalen Duren)]
8
Found Combo
total_projected_points 231.34999999999997
ttw 18.889999999999873
free agent Austin Reaves
dropping player Reggie Jackson
Made it
day of games 12/18/22
[Player(Kyle Kuzma), Player(O.G. Anunoby), Player(Russell Westbrook), Player(Scottie Barnes), Player(Bol Bol), Player(Royce O'Neale), Player(Kentavious Caldwell-Pope), Player(Austin Rea

{Player(Malik Beasley): {Player(Alec Burks): ['12/18/22', 22.230000000000018],
  Player(Jaden McDaniels): ['12/18/22', 22.12999999999988],
  Player(Markelle Fultz): ['12/18/22', 21.87999999999988],
  Player(Immanuel Quickley): ['12/18/22', 21.579999999999927],
  Player(Moritz Wagner): ['12/18/22', 21.34999999999991],
  Player(Aleksej Pokusevski): ['12/16/22', 21.200000000000045],
  Player(Jalen McDaniels): ['12/18/22', 21.190000000000055],
  Player(Nick Richards): ['12/18/22', 20.99000000000001],
  Player(Ayo Dosunmu): ['12/18/22', 20.480000000000018],
  Player(Grant Williams): ['12/18/22', 20.34999999999991],
  Player(Andre Drummond): ['12/18/22', 20.1099999999999],
  Player(Austin Reaves): ['12/18/22', 19.09999999999991],
  Player(Deni Avdija): ['12/18/22', 18.909999999999854],
  Player(Jalen Duren): ['12/18/22', 18.889999999999873],
  Player(Jaylen Nowell): ['12/18/22', 18.829999999999927],
  Player(Kyle Anderson): ['12/18/22', 18.159999999999854],
  Player(Jalen Williams): ['12/16/

In [ ]:
kuzma       sf/pf          pf
og          sf             sf
russ        pg             pg
scottie     sf/pf          f
clarkson    sg             sg
jonas       C              c    
bol         PF/C
oneale      SF/PF  
jackson     pg             g            
murray      pf

In [ ]:
tatum      sf/pf    
banchero   sf/pf     
vucevic    c         
Green      sg        
Sengun     c
Hart       SG/SF
Thompson   SG
washington pf/c      
boucher    pf/c
robinson   c
white      pg/sg     
hartenstein pf/c
           